# Práctica 5 de IA

## 1. Construcción de clasificadores en bases de datos sintéticas

In [ ]:
# Esta primera parte está adaptada de:
# http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons, make_circles, make_blobs
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

%matplotlib inline

ImportError: No module named sklearn.datasets

In [ ]:
# data preparation
simple = make_blobs(n_samples=500, n_features=2,
                    centers=[[0,0], [2.5,2.5]],
                    random_state=1)
X,y = make_blobs(n_samples=500, n_features=2,
                 centers=[[0,0], [5,1]],
                 random_state=1)
X = X*np.matrix([[1,-2],[-20,10]])
linearly_separable=(X,y)

datasets = [simple,
            linearly_separable,
            make_moons(noise=0.1, random_state=0, n_samples=500),
            make_circles(noise=0.1, factor=0.5, random_state=1,
                         n_samples=500)
           ]

In [ ]:
classifiers = [("Naive Bayes", GaussianNB()),
               ("Nearest Neighbors",
                KNeighborsClassifier(n_neighbors=1)), # número de vecinos
               ("Decision Tree",
                DecisionTreeClassifier(criterion='entropy',
                                       max_depth=2)), # profundidad máxima del árbol
               ("Logistic Regression",
                LogisticRegression(C=1e10)), # C: cuanto más alto menos regularización 
               ("Neural Network",
                MLPClassifier(hidden_layer_sizes=(50,), # tamaño capas ocultas
                              max_iter=1000, # número máximo de iteraciones
                              alpha=0))] # factor de regularización

from p5_IA_aux import plot_classifiers

plot_classifiers(classifiers, datasets)

In [ ]:
# Listado de los parámetros de los diferentes clasificadores
for name, clf in classifiers:
    print(clf)

Prueba a cambiar los siguientes parámetros y observa las consecuencias en la frontera de clasificación construida:

* Número de vecinos en k-nn: Prueba a ir subiendo este número. ¿Por qué siempre debe ser impar cuando hay dos clases?
* Profundidad máxima de los árboles de decisión: Prueba a ir subiendo este número.
* Factor de regularización (C) en la regresión logística: prueba C=1e10, C=1 y C=1e-7.
* Número de neuronas en la red neuronal y máximo número de épocas de entrenamiento. ** Nota: ** (50,) indica una única capa oculta con 50 neuronas. Prueba a reducir este número. También puedes probar a introducir más de una capa oculta. Por ejemplo, (50,10,) indica dos capas ocultas con 50 y 10 neuronas respectivamente; (50,10,20,) indica tres capas ocultas con 50, 10 y 20 neuronas respectivamente, etc.

# 2. Construcción de un clasificador en una base de datos real

In [ ]:
import pandas as pd
df = pd.read_csv("pima_construccion.csv", header=0, sep=',')
print(df.shape)
df.head(5)

* ** Pregnancies: ** Number of times pregnant
* ** Glucose: ** Plasma glucose concentration a 2 hours in an oral glucose tolerance test
* ** BloodPressure: ** Diastolic blood pressure (mm Hg)
* ** SkinThickness: ** Triceps skin fold thickness (mm)
* ** Insulin: ** 2-Hour serum insulin (mu U/ml)
* ** BMI: ** Body mass index (weight in kg/(height in m)^2)
* ** DiabetesPedigreeFunction: ** Diabetes pedigree function
* ** Age: ** Age (years)
* ** Class: ** Class variable (0 or 1)

In [ ]:
df.describe()

In [ ]:
#### Histogram Matrix Plot
fig = df.hist(figsize=(20, 10))
[x.title.set_size(18) for x in fig.ravel()]
plt.show()

In [ ]:
nombres_atrs = list(df.columns)
nombres_atrs.remove('class')
print(nombres_atrs)
X = df[nombres_atrs].values
y = df['class'].values

In [ ]:
### Histogramas suavizados con diferentes clases
for n in nombres_atrs:
    plt.figure(figsize=(3,2))
    df.groupby("class")[n].plot(kind='kde',
                                title='Hist. de '+n)
    plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

## Entrenamiento de un modelo y chequeo de su calidad usando 5-fold cross validation

La siguiente celda entrena un modelo y lo evalúa en varias particiones training-test diferentes de los datos. El resultado es un score medio junto a su desviación estándar. El tipo de modelo (Naïve Bayes / árbol de decisión / knn/ regresión logística / red neuronal) y parámetros empleados deberán ser seleccionados para que dicho resultado sea el mejor posible.

In [ ]:
clf = DecisionTreeClassifier(max_depth=3)
scores = cross_val_score(clf, X, y, cv=5)
print("scores: ", scores)
print("{:.2f} +/- {:.2f}".format(scores.mean(), scores.std()))

## Ajuste final del modelo usando todos los datos y predicción del conjunto de explotación

El modelo seleccionado en la celda anterior se ajustará ahora usando todos los datos etiquetados disponibles, y se usará para predecir datos no etiquetados (base de datos de "explotación"). Se guardarán dichas predicciones en el archivo de predicciones de explotación.

In [ ]:
clf.fit(X, y)

In [ ]:
df_explotacion = pd.read_csv("pima_explotacion.csv", header=0, sep=',')
df_explotacion.head(5)

In [ ]:
X_explotacion = df_explotacion[nombres_atrs].values
predicciones = clf.predict(X_explotacion)

f = open("predicciones_explotacion.txt", "w")
for pred in predicciones:
    f.write(str(pred)+"\n")
f.close()